### Importing Libraries 

In [ ]:
import pandas as pd
import re
import numpy as np 
from pandas import DataFrame
import time
from sklearn.feature_extraction.text import TfidfVectorizer

### Load Dataset

In [ ]:
client_name = pd.read_csv("xyz.csv", encoding = "ISO-8859-1")
left_side = client_name['Leftcol']

### Definig Ngram function and string cleaning and manipulation

In [ ]:
def ngrams(string, n=3):
    string = string.encode("ascii", errors="ignore").decpde()
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'","\""]
    rx = '['+ re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()
    string = re.sub(' +', ' ', string).strip() #combines white space
    string = ' ' + string + ' ' #padding
    string = re.sub(r'[,-./]|\sBD', r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

### Vectorizing string names and making matrix

In [ ]:
right_side = client_name['Rightcol']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix1 = vectorizer.fit_transform(right_side)

In [1]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tf_idf_matrix1)

#left_side = list(client_name['Leftcol'].values)
left_side = set(client_name['Leftcol'].values)

def getNearesetN(query):
    queryTFIDF = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF)
    return distances, indices

#calculating the time to return the value 
t1 = time.time()
print("getting nearest n...")
distances, indices = getNearestN(left_side)
t = time.time()-t1
print("completed IN : ", t)

#need more faster output convert into set it will removes duplicate records
#convert set to list for priting output

left_side = list(left_side)
print('organizing the output matches... ')
matches = []

for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), right_side.values[j],left_side[i]]
    matches = pd.DataFrame(matches, columns=['Match Confidence (lower is better)', 'rightside_matched_name','leftside_name'])
    print("done")

SyntaxError: EOL while scanning string literal (<ipython-input-1-a1aa08e2ef08>, line 29)

In [ ]:
print(matches)

In [ ]:
matches.to_csv('tf-idf_nearestN.csv', index = False)